# Начальная инициализация

In [1]:
import datetime
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:34:59


In [ ]:
PATH = '/kaggle/input/ml-3-lab/'
# PATH = ''

In [2]:
# Подключение библиотек

# !pip install torch torchvision torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils

# !pip install scikit-learn
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import ParameterGrid

import torch.optim as optim
# !pip install pandas
import pandas as pd
import numpy as np

# import re
# !pip install nltk
# import nltk
# nltk.download('punkt')
# from string import punctuation

# !pip install gensim
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

import os
import random

In [4]:
# Подключение вычислений на видеокарту, если доступна
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print(device)

cuda:0


In [5]:
cpu_count = os.cpu_count()

num_workers = cpu_count if device == torch.device("cpu") else 0
num_workers, cpu_count

(0, 4)

In [6]:
random_state = 42

torch.use_deterministic_algorithms(True)

%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%env PYTHONHASHSEED=42

def set_random_state(random_state):
    torch.manual_seed(random_state)
    random.seed(random_state)
    np.random.seed(random_state)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_state)
        torch.cuda.manual_seed(random_state)

        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
set_random_state(42)

env: CUBLAS_WORKSPACE_CONFIG=:4096:8
env: PYTHONHASHSEED=42


# Модели и методы для обучения и тестирования

## Предобработка текста

In [ ]:
train = pd.read_csv(PATH + 'X_y_train.csv', sep=';')
test = pd.read_csv(PATH + 'X_y_test.csv', sep=';')

X_train = [el[0] for el in train[['Text']].values]
X_test = [el[0] for el in test[['Text']].values]
y_train = [el[0] for el in train[['Class']].replace(-1, 0).values]
y_test = [el[0] for el in test[['Class']].replace(-1, 0).values]

# Create a X_token file one time!

# def tokenize(text):

#   text_token = nltk.word_tokenize(text)
#   text_word = [el.lower() for el in text_token if el not in punctuation]
#   return text_word

# X_train_token = [tokenize(t) for t in X_train]
# X_test_token = [tokenize(t) for t in X_test]


# with open('X_train_token.txt', mode='w+') as file:
#     for sentence in X_train_token:
#         print(*sentence, file=file)

X_token = [sentence.split() for sentence in open(PATH + 'X_token.txt', mode='r')]
X_train_token = [sentence.split() for sentence in open(PATH + 'X_train_token.txt', mode='r')]
X_test_token = [sentence.split() for sentence in open(PATH + 'X_test_token.txt', mode='r')]

In [ ]:
# !pip install pymorphy2 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
X_train_morph=[[str(morph.parse(w)[0].normal_form) for w in l] for l in X_train_token]
X_test_morph=[[str(morph.parse(w)[0].normal_form) for w in l] for l in X_test_token]

In [ ]:
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split

def build_vocabulary(data):
    # Builds the vocabulary and keeps the "x" most frequent words
    vocabulary = dict()
    fdist = nltk.FreqDist()
   
    for sentence in data:
        for word in sentence:
            fdist[word] += 1
          
    common_words = fdist.most_common(10000)
   
    for idx, word in enumerate(common_words):
        vocabulary[word[0]] = (idx+1)
     
    return vocabulary

Vocab = build_vocabulary(X_train_morph + X_test_morph)

def word_to_idx(data, vocabulary):
    # By using the dictionary (vocabulary), it is transformed
    # each token into its index based representation
 
       x_vocabulary = list()
    
       for sentence in data:
            temp_sentence = list()
            for word in sentence:
                if word in vocabulary.keys():
                    temp_sentence.append(vocabulary[word])
            x_vocabulary.append(temp_sentence)
     
       return  x_vocabulary

WordsId_train = word_to_idx(X_train_morph, Vocab)
WordsId_test = word_to_idx(X_test_morph, Vocab)


def padding_sentences(x_vocabulary, seq_len):
     # Each sentence which does not fulfill the required len
     # it's padded with the index 0
 
       pad_idx = 0
       x_padded = list()
    
       for sentence in x_vocabulary:
            
            if len(sentence) < seq_len:
                
                while len(sentence) < seq_len:
                    sentence.insert(len(sentence), pad_idx)
                x_padded.append(sentence)
            else:
                sentence1=sentence[:20]
                x_padded.append(sentence1)
            
  
       #x_padded = np.array(x_padded, dtype=object)
       return  x_padded


    
WordsIdPad_train = padding_sentences(WordsId_train, 20)
X_test = padding_sentences(WordsId_test, 20)


X_train, X_val, y_train, y_val = train_test_split(WordsIdPad_train, y_train, test_size=0.2, random_state=42)

**ДЛЯ ОБУЧЕНИЯ НА УЧЕБНОМ ПК!!!**

In [7]:
import pickle
from sklearn.model_selection import train_test_split
PATH = 'dumps/'

with open(PATH + 'WordsIdPad_train.pkl', mode='rb') as file:
    WordsIdPad_train = pickle.load(file)

with open(PATH + 'WordsIdPad_test.pkl', mode='rb') as file:
    X_test = pickle.load(file)

with open(PATH + 'y_train.pkl', mode='rb') as file:
    y_train = pickle.load(file)

with open(PATH + 'y_test.pkl', mode='rb') as file:
    y_test = pickle.load(file)

X_train, X_val, y_train, y_val = train_test_split(WordsIdPad_train, y_train, test_size=0.2, random_state=42)

## Обучение нейронных сетей

In [11]:

param_grid = {
#     "layers": [
#         [32, 16, 8, 4]
#     ],
    "activation_fn": [F.relu, F.leaky_relu],
    "learning_rate": [0.001, 0.01],
    "epochs": [50, 20, 10, 5],
    "optimizer": [optim.SGD, optim.AdamW],
    "batch_size": [2048],
    "layers_count": [i for i in range(6, 0, -1)],
    "kernel_size": [3, 4, 5],
    "stride": [1, 2, 3]
}

params_list = ParameterGrid(param_grid)
len(params_list)

1728

In [12]:
class Net(nn.Module):
    def __init__(self, device=None, layers_count=None, activation_function=None, kernel=None, stride=None):
        super().__init__()
        
        self.num_words = 10000
        self.embedding_size = 300
        
        self.seq_len = 20
        self.out_size = 32
        
        self.fa = activation_function
        
        self.kernel = kernel
        
        self.stride = stride
        
        self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0, device=device)
        
        self.layers = nn.ModuleList([])
        
        output_shape = self.seq_len
        
        for i in range(layers_count):
            self.layers.append(nn.Conv1d(in_channels = self.embedding_size, out_channels = self.embedding_size, kernel_size = self.kernel, stride = self.stride, padding = 5, device=device))
            output_shape = 1 + (output_shape + 2*5 - self.kernel) // self.stride
        
        
        self.fc = nn.Linear(output_shape * self.embedding_size, 2, device=device)
        
    def forward(self, x):
        
        x = self.embedding(x)
        x = x.reshape(len(x), self.embedding_size, self.seq_len)
        
        for conv_layer in self.layers:
            x = conv_layer(x)
            x = self.fa(x)
        
        x = self.fc(x.reshape(x.shape[0], -1))
        
        return F.log_softmax(x, dim=-1)

In [13]:
from torchmetrics.classification import Accuracy, F1Score 
from tqdm import tqdm

total = []

# accuracy_criterion = Accuracy(task='multiclass', num_classes=2, multidim_average='global', average='weighted').to(device)
f1_criterion = F1Score(task='multiclass', num_classes=2, multidim_average='global', average='weighted').to(device)

# file = open('logs.txt', mode='w+')

for params in tqdm(params_list):
    
    learningRate = params['learning_rate']
    optimizer_type = params['optimizer']
    layers_count = params['layers_count']
    batch_size = params['batch_size']
    epochs = params['epochs']
    fa = params['activation_fn']
    kernel_size = params['kernel_size']
    stride = params['stride']
    

    net = Net(device=device, layers_count=layers_count, activation_function=fa, kernel=kernel_size, stride=stride).to(device)
    optimizer = optimizer_type(net.parameters(), lr=learningRate)
    
    
    features_train = torch.tensor(X_train, device=device, dtype=torch.int32)
    targets_train = torch.tensor(y_train, device=device, dtype=torch.int32)
    
    features_val = torch.tensor(X_val, device=device, dtype=torch.int32)
    targets_val = torch.tensor(y_val, device=device, dtype=torch.int32)
    
    features_test = torch.tensor(X_test, device=device, dtype=torch.int32)
    targets_test = torch.tensor(y_test, device=device, dtype=torch.int32)
    
    trainset = data_utils.TensorDataset(features_train, targets_train)
    valset = data_utils.TensorDataset(features_val, targets_val)
    testset = data_utils.TensorDataset(features_test, targets_test)
    
    train_loader = data_utils.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = data_utils.DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_loader = data_utils.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    for epoch in range(epochs):
        net.train()
        
        for X, Y in train_loader:
            output = net(X)
            loss = F.cross_entropy(output, Y.long())
            
            net.zero_grad()
            
            loss.backward()
            optimizer.step()
            
    
    train_f1 = 0
    val_f1 = 0
    test_f1 = 0
    
    with torch.no_grad():
        net.eval()
        
        # evaluate the model on train
        targets = torch.tensor([], device=device)
        predictions = torch.tensor([], device=device)

        for X, Y in train_loader:
            output = net(X)
            
            targets = torch.cat((targets, Y))
            predictions = torch.cat((predictions, output.argmax(dim=-1)))
        
        train_f1 = f1_criterion(predictions, targets).item()

        # evaluate the model on validation
        targets = torch.tensor([], device=device)
        predictions = torch.tensor([], device=device)

        for X, Y in val_loader:
            output = net(X)
            
            targets = torch.cat((targets, Y))
            predictions = torch.cat((predictions, output.argmax(dim=-1)))
        
        val_f1 = f1_criterion(predictions, targets).item()

        # evaluate the model on test
        targets = torch.tensor([], device=device) 
        predictions = torch.tensor([], device=device)

        for X, Y in test_loader:
            output = net(X)
            
            targets = torch.cat((targets, Y))
            predictions = torch.cat((predictions, output.argmax(dim=-1)))
        
        test_f1 = f1_criterion(predictions, targets).item()
            
        
    
    total.append({
        'Функция активации': fa.__name__,
        'Размер ядра': kernel_size,
        "Отступ": stride,
        "Скорость обучения": learningRate,
        'Количество слоев': layers_count,
        'Batch_size': batch_size,
        'Оптимизатор': type(optimizer).__name__,
        'Количество эпох': epochs,
        'F1-train': round(train_f1, 2),
        'F1-val': round(val_f1, 2),
        'F1-test': round(test_f1, 2)
    })


100%|██████████| 1728/1728 [14:33:37<00:00, 30.33s/it]    


In [2]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

26-May-2023 18:35:33


# Сводная таблица

In [15]:
pd.set_option('display.max_rows', None)
summary = pd.DataFrame.from_dict(total)
summary_sort = summary.sort_values(by='F1-val', ascending=False)
summary_sort

,Функция активации,Размер ядра,Отступ,Скорость обучения,Количество слоев,Batch_size,Оптимизатор,Количество эпох,F1-train,F1-val,F1-test
442,relu,3,2,0.010,6,2048,AdamW,10,0.99,0.72,0.76
285,relu,3,1,0.010,1,2048,AdamW,20,1.00,0.72,0.68
286,relu,3,2,0.010,1,2048,AdamW,20,1.00,0.72,0.64
670,relu,3,2,0.010,5,2048,AdamW,5,0.89,0.72,0.68
1221,leaky_relu,4,1,0.010,1,2048,AdamW,20,1.00,0.72,0.78
1365,leaky_relu,3,1,0.010,1,2048,AdamW,10,1.00,0.72,0.72
1546,leaky_relu,3,2,0.010,4,2048,AdamW,5,0.89,0.71,0.74
1293,leaky_relu,5,1,0.010,1,2048,AdamW,20,1.00,0.71,0.68
274,relu,3,2,0.010,2,2048,AdamW,20,1.00,0.71,0.70
1353,leaky_relu,3,1,0.010,2,2048,AdamW,10,1.00,0.71,0.76


# Выводы

Как можно увидеть по таблице - самого лучшего результата (0.80) достигла модель c оптимизатором AdamW, с размером ядра 4 и отступом 1. Для этого потребовалось 10 эпох обучения и 2 сверточных слоя. Данный результат похож на результат (0.84), который был показан с сетями прямого распространения.